# Model Deployment

**Project Goals**:

In [1]:
%load_ext autoreload
%autoreload

import os
import sqlite3
import pandas as pd
import requests
from arch.univariate.base import ARCHModelResult
from config import settings
from data import SQLRepository

d:\workspace\applied-data_science\venv\Lib\site-packages\pydantic\_internal\_fields.py:160: UserWarning: Field "model_directory" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(


#### Connect to Database

In [2]:
connection = sqlite3.connect(settings.db_name, check_same_thread=False)
repo = SQLRepository(connection=connection)

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

repo type: <class 'data.SQLRepository'>
repo.connection type: <class 'sqlite3.Connection'>


#### GarchModel Class: Init Method Assertion

In [3]:
from model import GarchModel

# Instantiate a `GarchModel`
gm_ambuja = GarchModel(ticker="AMBUJACEM.BSE", repo=repo, use_new_data=False)

# Does `gm_ambuja` have the correct attributes?
assert gm_ambuja.ticker == "AMBUJACEM.BSE"
assert gm_ambuja.repo == repo
assert not gm_ambuja.use_new_data
assert gm_ambuja.model_directory == settings.model_directory

#### GarchModel Class: Wrangle_data Method

In [4]:
# Instantiate `GarchModel`, use new data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=True)

# Check that model doesn't have `data` attribute yet
assert not hasattr(model_shop, "data")

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Does model now have `data` attribute?
assert hasattr(model_shop, "data")

# Is the `data` a Series?
assert isinstance(model_shop.data, pd.Series)

# Is Series correct shape?
assert model_shop.data.shape == (1000,)

model_shop.data.head()

date
2020-06-04   -4.979147
2020-06-05    4.655099
2020-06-08   -2.165788
2020-06-09   -4.784586
2020-06-10   -4.749985
Name: return, dtype: float64

#### GarchModel Class: Fit Method

In [5]:
# Instantiate `GarchModel`, use old data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=False)

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Fit GARCH(1,1) model to data
model_shop.fit(p=1, q=1)

# Does `model_shop` have a `model` attribute now?
assert hasattr(model_shop, "model")

# Is model correct data type?
assert isinstance(model_shop.model, ARCHModelResult)

# Does model have correct parameters?
assert model_shop.model.params.index.tolist() == ["mu", "omega", "alpha[1]", "beta[1]"]

# Check model parameters
model_shop.model.summary()

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2311.0340670585474
            Iterations: 15
            Function evaluations: 92
            Gradient evaluations: 15


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2311.03
Distribution:                  Normal   AIC:                           4630.07
Method:            Maximum Likelihood   BIC:                           4649.70
                                        No. Observations:                 1000
Date:                Sun, Jun 09 2024   Df Residuals:                      999
Time:                        17:04:41   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.1309  7.128e-02      1.836  6.634e-02 [-8.829e-03,  0.271]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0107  1.562e-02      0.684      0.494 [-1.993e-02,4.130e-02]
alpha[1]   8.2658e-03  5.447e-03      1.517      0.129 [-2.410e-03,1.894e-02]
beta[1]        0.9891  6.255e-03    158.118      0.000      [  0.977,  1.001]
=============================================================================

Covariance estimator: robust
"""

### GarchModel Class: Predict_volatility Method Assertions

In [6]:
# Generate prediction from `model_shop`
prediction = model_shop.predict_volatility(horizon=5)

# Is prediction a dictionary?
assert isinstance(prediction, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction.keys())

# Are values correct data type?
assert all(isinstance(v, float) for v in prediction.values())

prediction

{'2024-06-10T00:00:00': 1.8303930188668602,
 '2024-06-11T00:00:00': 1.830861755996761,
 '2024-06-12T00:00:00': 1.8313291188583267,
 '2024-06-13T00:00:00': 1.8317951118602422,
 '2024-06-14T00:00:00': 1.8322597393946447}

#### GarchModel Class: Dump Method

In [7]:
# Save `model_shop` model, assign filename
filename = model_shop.dump()

# Is `filename` a string?
assert isinstance(filename, str)

# Does filename include ticker symbol?
assert model_shop.ticker in filename

# Does file exist?
assert os.path.exists(filename)

filename

OSError: [Errno 22] Invalid argument: 'models\\2024-06-09T17:04:41.731514_SHOPERSTOP.BSE.pkl'

#### Load Function

In [ ]:
def load(ticker):

    """Load latest model from model directory.

    Parameters
    ----------
    ticker : str
        Ticker symbol for which model was trained.

    Returns
    -------
    `ARCHModelResult`
    """
    # Create pattern for glob search
    pattern = os.path.join(settings.model_directory, f"*{ticker}.pkl")

    # Try to find path of latest model
   
    try:
        model_path = sorted(glob(pattern))[-1]
  # Handle possible `IndexError`
    except IndexError:
        raise Exception(f"No module with {ticker}.")    
    # Load model
    model = joblib.load(model_path)

    # Return model
    return model

In [ ]:
# Assign load output to `model`
model_shop = load(ticker="SHOPERSTOP.BSE")

# Does function return an `ARCHModelResult`
assert isinstance(model_shop, ARCHModelResult)

# Check model parameters
model_shop.summary()

#### GarchModel Class: Load Method

In [ ]:
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=False)

# Check that new `model_shop_test` doesn't have model attached
assert not hasattr(model_shop, "model")

# Load model
model_shop.load()

# Does `model_shop_test` have model attached?
assert hasattr(model_shop, "model")
model_shop.model.summary()